# Setup imports 

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import os

import caiman as cm
from caiman.paths import caiman_datadir
from caiman.source_extraction.cnmf import cnmf, params
from caiman.source_extraction.cnmf.oasis import oasisAR1, OASIS
from caiman.source_extraction.cnmf.deconvolution import estimate_parameters, axcov, constrained_foopsi

In [ ]:
def gen_data(g=[.95], sn=.3, T=3000, framerate=30, firerate=.5, b=0, seed=0, sinusoidal=False):
    """
    Generate data from homogenous Poisson Process

    Parameters
    ----------
    g : array, shape (p,), optional, default=[.95]
        Parameter(s) of the AR(p) process that models the fluorescence impulse response.
    sn : float, optional, default .3
        Noise standard deviation.
    T : int, optional, default 3000
        Duration.
    framerate : int, optional, default 30
        Frame rate.
    firerate : int, optional, default .5
        Neural firing rate.
    b : int, optional, default 0
        Baseline.
    seed : int, optional, default 0
        Seed of random number generator.
    sinusoidal : bool, optional, default False
        Generate sinusoidal data, i.e. a spike train with significant autocorrelation

    Returns
    -------
    y : array, shape (T,)
        Noisy fluorescence data.
    c : array, shape (T,)
        Calcium traces (without sn).
    s : array, shape (T,)
        Spike trains.
    """

    np.random.seed(seed)
    Y = np.zeros(T)
    if sinusoidal:
        s = np.random.rand(T) < firerate / float(framerate) * np.sin(np.arange(T) // 50)**3 * 4
    else:
        s = np.random.rand(T) < firerate / float(framerate)    
    c = s.astype(float)
    for i in range(2, T):
        if len(g) == 2:
            c[i] += g[0] * c[i-1] + g[1] * c[i-2]
        else:
            c[i] += g[0] * c[i-1]
    y = b + c + sn * np.random.randn(T)
    return y.astype(np.float32), c, s

In [ ]:
def plot_trace(groundtruth=False):
    plt.figure(figsize=(20,4))
    plt.subplot(211)
    plt.plot(b+c, lw=2, label='denoised', c='#0072B2')
    if groundtruth:
        plt.plot(true_b+true_c, c='#D55E00', label='truth', zorder=-11)
    plt.plot(y, label='data', zorder=-12, c='#999999')
    plt.legend(ncol=3, frameon=False, loc=(.02,.85))
    plt.subplot(212)
    plt.plot(s, lw=2, label='deconvolved', c='#009E73')
    if groundtruth:
        for k in np.where(true_s)[0]:
            plt.plot([k,k],[-.1,1], c='#D55E00', zorder=-11, clip_on=False)
    plt.ylim(0,1.3)
    plt.legend(ncol=3, frameon=False, loc=(.02,.85));
    print("     Correlation of deconvolved activity  with ground truth ('spikes') : %.4f" % np.corrcoef(s,true_s)[0,1])
    print("     Correlation of denoised fluorescence with ground truth ('calcium'): %.4f" % np.corrcoef(c,true_c)[0,1])
    kern = np.exp(-np.arange(-15,16)**2/8)
    print("Schreiber metric of deconvolved activity  with ground truth ('spikes') : %.4f" % np.corrcoef(np.convolve(s, kern), np.convolve(true_s, kern))[0,1])

# Load raw fluorescence data 
This data happend to have a fast rise time and we model it with an AR(1) process, see below for slower rise time and AR(2).

In [ ]:
# here we generate some simulated fluorescence data and plot it
true_b = 2
y, true_c, true_s = gen_data(b=true_b)
plt.figure(figsize=(20,4))
plt.plot(y, c='#999999')
plt.xlabel('Frames')
plt.ylabel('Fluorescence');

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(true_s[80:250], c='#D55E00')
plt.plot(true_c[80:250], c='#0072B2')
plt.xlabel('Frames')
plt.ylabel('Fluorescence');

# Estimate noise level and decay factor

In [ ]:
g, sn = estimate_parameters(y, 1)
g, sn

In [ ]:
ff, Pxx = scipy.signal.welch(y)
ind = (.25 < ff) * (ff < .5)

plt.figure(figsize=(18,4))
plt.subplot(131)
plt.semilogy(ff, Pxx/2)
plt.semilogy(ff[ind], Pxx[ind]/2)
plt.axhline(sn**2, c='C2')
plt.xlabel('Freq / Nyquistfreq')
plt.ylabel('Power spectral density')
plt.text(.02, sn**2*1.1, '$\sigma^2$', c='C2', size=20)

lags = 15
rng = range(-lags,lags+1)
ac = axcov(y,lags)
ac2 = ac.copy()
ac2[lags] -= sn**2

for k,plf in ((2, plt.plot), (3, plt.semilogy)):
    plt.subplot(1,3,k)
    plf(rng,ac)
    plf(rng,ac2)
    plf(rng, .168*g**np.abs(range(-lags,lags+1)),c='r')
    plf([3,3], [ac[lags], ac2[lags]], lw=3)
    plt.text(4, (ac[lags] + ac2[lags])/2, '$\sigma^2$', c='C2', size=20)
    plt.xlabel('Lag')
    plt.ylabel('Autocovariance');
plt.tight_layout()

### $g$ is related to the decay time $\tau_d$ of the exponetial Ca response kernel $e^{-t/\tau_d}$ as $g=e^{-\frac{1}{\tau_d r}}$ with decay time $\tau_d$ in seconds and imaging rate $r$ in Hz.

In [ ]:
# exponential decay time in seconds
td = -1/np.log(.95)/30
td

In [ ]:
# half decay time in seconds
thalf = td *np.log(2)
thalf

In [ ]:
.95**(thalf*30)

In [ ]:
t = np.linspace(0,2)
plt.figure(figsize=(6,4))
plt.plot(t, np.exp(-t/td), c='#0072B2', label=r'$e^{-t/\tau_d}$')
plt.plot(t, .95**(t*30), ':', c='k', label='$g^{t*r}$')
plt.legend()
plt.axvline(0, c='#D55E00')
plt.plot([0,thalf], [.5,.5], c='C2')
plt.text(thalf/10, .52, r'$\tau_{1/2}$', c='C2', size=20)
plt.plot([0,td], [np.exp(-1)]*2, c='C2')
plt.text(td/3, 1.06*np.exp(-1), r'$\tau_d$', c='C2', size=20)
plt.xlabel('Time [s]')
plt.ylabel('Fluorescence [au]');
plt.title(r'$e^{-t/\tau_d}$ = $g^{t*r}$');

# Deconvolve 

### With $\ell_1$ penalty we obtain the global minimum of the convex problem 
If we only have the trace and no further info, simply calling *constrained_foopsi* tries to estimate it from the data 

In [ ]:
%time c, b, c1, g, sn, s, lam = constrained_foopsi(y, p=1)

#### Plot results
Because we happen to have ground truth data, we show it too 

In [ ]:
plot_trace(True)

### With minimum spike size $s_{min}$ the problem is non-convex, however, we obtain a good local minimum

In [ ]:
# If s_min=0 the threshold is determined automatically such that RSS <= sn^2 T
%time c, b, c1, g, sn, s, lam = constrained_foopsi(y, p=1, s_min=0)
plot_trace(True)

In [ ]:
# For positive values the threshold is s_min
%time c, b, c1, g, sn, s, lam = constrained_foopsi(y, p=1, s_min=.55)
plot_trace(True)

In [ ]:
# For negative values the threshold is abs(s_min) * sn * sqrt(1-g)
%time c, b, c1, g, sn, s, lam = constrained_foopsi(y, p=1, s_min=-5)
plot_trace(True)

### If we have a good idea about the fluorescence baseline, its time constant and it size for 1AP, we can provide this information
$g$ is related to the decay time $\tau_d$ of the exponetial Ca response kernel $e^{-t/\tau_d}$ as $g=e^{-\frac{1}{\tau_d r}}$ with decay time $\tau_d$ in seconds and imaging rate r in Hz.

In [ ]:
# Here we provide the ground truth values for b and g. 
# The Ca response kernel to 1 AP has maximal amplitude 1 and we pick s_min slightly larger than 1/2.
%time c, s = oasisAR1(y-true_b, g=.95, s_min=.55)
plot_trace(True)

### If $g$ is not provided, it is estimated from the autocorrelation. We can improve upon it, in particularly if the spiking signal has some significant autocorrelation.
In function deconvolve, we already multiply the autocorrelation estimate by some fudge_factor that is close to but smaller than 1, which increases robustness

In [ ]:
y, true_c, true_s = gen_data(b=true_b, sinusoidal=True)

In [ ]:
# no optimization of g, fudge_factor=1
%time c, b, c1, g, sn, s, lam  = constrained_foopsi(y, p=1)
plot_trace(True)
plt.show()
print('Estimated decay factor gamma %.3f' % g)

In [ ]:
# no optimization of g, fudge_factor=.99
%time c, b, c1, g, sn, s, lam  = constrained_foopsi(y, p=1, fudge_factor=.99) 
plot_trace(True)
plt.show()
print('Estimated decay factor gamma %.3f' % g)

In [ ]:
# optimization of g. optimize_g=5 uses 5 large isolated calcium events to update g.
%time c, b, c1, g, sn, s, lam  = constrained_foopsi(y, p=1, optimize_g=5, fudge_factor=.99, s_min=-5)
plot_trace(True)
plt.show()
print('Estimated decay factor gamma %.3f' % g)

# Load some other raw fluorescence data 
This data happend to have a slow rise time and we model it with an AR(2) process

In [ ]:
# here we generate some simulated fluorescence data and plot it
true_g = [1.7,-.712]
y, true_c, true_s = gen_data(true_g, sn=.5, b=true_b)
plt.figure(figsize=(20,4))
plt.plot(y, c='#999999')
plt.xlabel('Frames')
plt.ylabel('Fluorescence');

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(true_s[80:250], c='#D55E00')
plt.plot(true_c[80:250]/3, c='#0072B2')
plt.xlabel('Frames')
plt.ylabel('Fluorescence');

# Estimate noise level and decay factor

In [ ]:
g, sn = estimate_parameters(y, 2)
g, sn

### $g=(g_1,g_2)$ is related to the decay time $\tau_d$ and rise time $\tau_r$ (in seconds) of the Ca response kernel $e^{-t/\tau_d}-e^{-t/\tau_r}$ as <br> $g_1=e^{-\frac{1}{\tau_d r}}+e^{-\frac{1}{\tau_r r}}$ and <br>$g_2=-e^{-\frac{1}{\tau_d r}}\cdot e^{-\frac{1}{\tau_r r}}$ with imaging rate r in Hz.

In [ ]:
d = (true_g[0] + np.sqrt(true_g[0] * true_g[0] + 4 * true_g[1])) / 2
r = (true_g[0] - np.sqrt(true_g[0] * true_g[0] + 4 * true_g[1])) / 2
d, r

In [ ]:
d = (g[0] + np.sqrt(g[0] * g[0] + 4 * g[1])) / 2
r = (g[0] - np.sqrt(g[0] * g[0] + 4 * g[1])) / 2
d, r

In [ ]:
d = (g[0] + np.sqrt(g[0] * g[0] + 4 * g[1])) / 2
r = (g[0] - np.sqrt(g[0] * g[0] + 4 * g[1])) / 2
d, r

In [ ]:
# exponential decay and rise time in seconds
td = -1/np.log(d)/30
tr = -1/np.log(r)/30
td, tr

In [ ]:
tpeak = np.log(tr/td) / (1/td-1/tr)

In [ ]:
t = np.linspace(0,2,200)
plt.figure(figsize=(6,4))
plt.plot(t, np.exp(-t/td)-np.exp(-t/tr), c='#0072B2', label=r'$e^{-t/\tau_d}-e^{-t/\tau_r}$')
plt.plot(t, d**(t*30), ':', c='#0072B2', label=r'$e^{-t/\tau_d}$')
plt.plot(t, 1-r**(t*30), ':', c='#0072B2', label=r'$1-e^{-t/\tau_r}$')
plt.legend()
plt.axvline(0, c='#D55E00')
plt.plot([0,td], [np.exp(-1)]*2, c='C2')
plt.text(td/3, 1.06*np.exp(-1), r'$\tau_d$', c='C2', size=20)
plt.plot([0,tr], [1-np.exp(-1)]*2, c='C2')
plt.text(tr/3, 1.06*(1-np.exp(-1)), r'$\tau_r$', c='C2', size=20)
plt.xlabel('Time [s]')
plt.ylabel('Fluorescence [au]');
plt.title(r'$e^{-t/\tau_d}$ = $g^{t*r}$');

# Deconvolve 

### With $\ell_1$ penalty we obtain the global minimum of the convex problem 
If we only have the trace and no further info, simply calling deconvolve tries to estimate it from the data 

In [ ]:
# modelling by an AR(1) process, i.e w/o rise time, ...
%time c, b, c1, g, sn, s, lam = constrained_foopsi(y, p=1)
plot_trace(True)

In [ ]:
# ... is outperformed by an AR(2) process
%time c, b, c1, g, sn, s, lam = constrained_foopsi(y, p=2)
plot_trace(True)

### With minimum spike size $s_{min}$ the problem is non-convex, however, we obtain a good local minimum

In [ ]:
# For negative values the threshold is abs(s_min) * sn * sqrt(1-d)
%time c, b, c1, g, sn, s, lam = constrained_foopsi(y, p=2, s_min=-5)
plot_trace(True)
g

### If we have a good idea about the fluorescence time constants we can provide this information
$g=(g_1,g_2)$ is related to the decay time $\tau_d$ and rise time $\tau_r$ (in seconds) of the Ca response kernel $e^{-t/\tau_d}-e^{-t/\tau_r}$ as <br>
$g_1=e^{-\frac{1}{\tau_d r}}+e^{-\frac{1}{\tau_r r}}$ and <br>
$g_2=-e^{-\frac{1}{\tau_d r}}\cdot e^{-\frac{1}{\tau_r r}}$ with imaging rate r in Hz.

In [ ]:
%time c, b, c1, g, sn, s, lam = constrained_foopsi(y, p=2, g=[1.7,-.712])
plot_trace(True)

In [ ]:
# For negative values the threshold is abs(s_min) * sn * sqrt(1-d)
%time c, b, c1, g, sn, s, lam = constrained_foopsi(y, p=2, g=[1.7,-.712], s_min=-4)
plot_trace(True)

# Online processing 

In [ ]:
%matplotlib notebook

# generate object
o = OASIS(g=true_g[0], lam=0, s_min=.5, b=true_b, g2=true_g[1])

fig = plt.figure(figsize=(12,4))
plt.subplot(211)
tmp, tmp2, tmp3 = np.nan*np.ones((3,3000))
line1, = plt.plot(tmp, c='#999999')
line2, = plt.plot(tmp, c='#0072B2')
plt.xlim(0,3000)
plt.ylim(-1,y.max()-true_b)
plt.subplot(212)
line3, = plt.plot(tmp, c='#009E73')
plt.xlim(0,3000)
plt.ylim(-.1,1.3)
plt.tight_layout()

for k,yt in enumerate(y):
    o.fit_next(yt) # fit next data point
    
    tmp2[:k+1] = o.c
    tmp3[:k+1] = o.s
    tmp[k] = yt-true_b
    line1.set_ydata(tmp)
    line2.set_ydata(tmp2)
    line3.set_ydata(tmp3)
    fig.canvas.draw()

In [ ]:
%matplotlib inline

# Interpreting CaImAn results 

#### Run basic demo with p=1

In [ ]:
%%capture

# %% start a cluster
c, dview, n_processes =\
    cm.cluster.setup_cluster(backend='local', n_processes=None,
                             single_thread=False)

# %% set up some parameters
fnames = [os.path.join(caiman_datadir(), 'example_movies', 'demoMovie.tif')]
                        # file(s) to be analyzed
fr = 10                 # approximate frame rate of data
decay_time = 5.0        # length of transient
rf = 10             # half size of each patch
stride = 4          # overlap between patches
K = 4               # number of components in each patch
gSig = [6, 6]           # expected half size of neurons
merge_thresh = 0.80     # merging threshold, max correlation allowed
p = 1                   # order of the autoregressive system
gnb = 2                 # global background order

params_dict = {'fnames': fnames,
               'fr': fr,
               'decay_time': decay_time,
               'rf': rf,
               'stride': stride,
               'K': K,
               'gSig': gSig,
               'merge_thr': merge_thresh,
               'p': p,
               'nb': gnb}

opts = params.CNMFParams(params_dict=params_dict)
# %% Now RUN CaImAn Batch (CNMF)
cnm = cnmf.CNMF(n_processes, params=opts, dview=dview)
cnm = cnm.fit_file()

# %% load memory mapped file
Yr, dims, T = cm.load_memmap(cnm.mmap_file)
images = np.reshape(Yr.T, [T] + list(dims), order='F')

# %% refit
cnm2 = cnm.refit(images, dview=dview)

# %% COMPONENT EVALUATION
min_SNR = 2      # peak SNR for accepted components (if above this, acept)
rval_thr = 0.85     # space correlation threshold (if above this, accept)
use_cnn = True      # use the CNN classifier
min_cnn_thr = 0.99  # if cnn classifier predicts below this value, reject
cnn_lowest = 0.1 # neurons with cnn probability lower than this value are rejected

cnm2.params.set('quality', {'min_SNR': min_SNR,
                            'rval_thr': rval_thr,
                            'use_cnn': use_cnn,
                            'min_cnn_thr': min_cnn_thr,
                            'cnn_lowest': cnn_lowest})

cnm2.estimates.evaluate_components(images, cnm2.params, dview=dview)

In [ ]:
def myplot(e, n=0):
    plt.figure(figsize=(20,4))
    plt.subplot(211)
    plt.plot(e.C[n]+e.YrA[n], label='raw', c='#999999')
    plt.plot(e.C[n], label='denoised', c='#0072B2')
    plt.axhline(e.bl[n], ls=':', c='C2')
    plt.subplot(212)
    plt.plot(e.S[n], label='deconvolved', c='#009E73')
    if len(e.g[n]) ==1:
        print('AR coefficients g: %.4f' % tuple(e.g[n]))
    else:
        print('AR coefficients g: %.4f,  %.4f' % tuple(e.g[n]))
    print('      baseline bl: %.4f' % e.bl[n])
    print('   initial fluor.: %.4f' % e.c1[n])
myplot(cnm2.estimates)

In [ ]:
# estimates.C includes the baseline estimates.bl, thus can be negative if bas_nonneg=False
cnm2.estimates.C[0].min()

#### Run deconvolution again but with p=2

In [ ]:
cnm3=cnm2
cnm3.deconvolve(p=2)
myplot(cnm3.estimates)

In [ ]:
# %% STOP CLUSTER
cm.stop_server(dview=dview)

#### Whats's the baseline $F_0$ for calculating $\Delta F/F_0$?

In [ ]:
cnm3.estimates.detrend_df_f()

plt.figure(figsize=(20,2))
plt.plot(cnm3.estimates.F_dff[0]);

In [ ]:
# B = A' * b*f for ||A||=1
nA = np.array(np.sqrt(cnm3.estimates.A.power(2).sum(0)))
B = cnm3.estimates.A.T.dot(cnm3.estimates.b).dot(cnm3.estimates.f)
nA

In [ ]:
plt.figure(figsize=(20,2))
F = cnm3.estimates.C[0] + cnm3.estimates.YrA[0] + B[0]
plt.plot(F, label='F')
plt.plot(B[0], label='B')
plt.fill_between(range(len(F)), F, B[0], color='C3', alpha=.3, label='$\Delta$ F')
plt.plot(scipy.ndimage.percentile_filter(F, 50, (500)), label='F0')
plt.legend();